# Virtual Drawing Pad

## Introductions

* In this project, I have made a Virtual Drawing Pad using a Stylus for drawing. The Stylus used is an object of **uniform, single tone** which will be hovered in front of the camera and used as a pen.
>
* In my demonstration I have used a bottle cap as Stylus.
>
* The code has been explained in blocks and markdowns and comments will explain what the block is doing.

## The Code

### 1. Importing the necessary libraries

In [45]:
import numpy as np
import cv2 as cv

### 2. Making a Trackbar window program to help the user find the color of the object.

* The user has to match the color of their objects with the output shown in the window by using the trackbars to adjust the R, G, B values.
>
* The values of the R, G, B values will be stored and used in finding the HSV values of the given color.

In [46]:
#Trackbar

def nothing(x):
    pass
# Create a black image, a window for displaying the color shade
img = np.zeros((300,512,3), np.uint8)
cv.namedWindow('Color Match')

# Create trackbars for color change

cv.createTrackbar('R','Color Match',0,255,nothing)
cv.createTrackbar('G','Color Match',0,255,nothing)
cv.createTrackbar('B','Color Match',0,255,nothing)

r,g,b=0,0,0 
#Declaring variables outside so it would be used in later blocks as well

while(1):
    cv.imshow('Color Match',img)
    k = cv.waitKey(1) & 0xFF
    if k == 27: #On pressing the `esc` key, the program closes
        break
    # get current positions of the three trackbars
    r = cv.getTrackbarPos('R','Color Match')
    g = cv.getTrackbarPos('G','Color Match')
    b = cv.getTrackbarPos('B','Color Match')
    
    #Display the color values as image
    img[:] = [b,g,r]
cv.destroyAllWindows()

### 3. Making another Trackbar window for adjusting HSV values

* This step is done as HSV values are sensitive to the lighting conditions.
>
* So just matching the color of the object won't be enough, the adjustments of the HSV is necessary so that the background doesn't uneccessarily appear in the camera and interfere with the execution.
>
* So the user has to basicaly move the object around all over, from the middle to the edges, and see if the circle detecting the object properly. If not, then the Lower and Upper Values of HSV trackbars can be adjusted suiting the lighting conditions to get desirable results
>
* So the user has to basically move the object around the screen all over, from the middle to the edges and see if the shape on the mask is prominent or is properly forming and no background is getting detected on motion. If not, then the Lower and Upper Values of HSV trackbars can be adjusted suiting the lighting conditions to get desirable results
>
>### **[NOTE]**:
>
>* The values of the Lower and Upper HSV is set to certain values by default for convenience, the user has to adjust only to the values nearby to the pre-set values if not getting the desirable result.

In [47]:
#HSV adjustment Trackbar

cap = cv.VideoCapture(0)

cv.namedWindow('Adjustment')

#Converting the Color obtained prevviously and converting it to it's respective HSV.
[[[u,v,w]]]= cv.cvtColor(np.uint8([[[b,g,r]]]),cv.COLOR_BGR2HSV)

# creating trackbars for HSV adjustment (which have a default value based on the color they chose.)

#For the lower HSV parameters
cv.createTrackbar('low H','Adjustment',u-20,255,nothing)
cv.createTrackbar('low S','Adjustment',50,255,nothing)
cv.createTrackbar('low V','Adjustment',100,255,nothing)

#For the upper HSV parameters
cv.createTrackbar('up H','Adjustment',u+10,255,nothing)
cv.createTrackbar('up S','Adjustment',255,255,nothing)
cv.createTrackbar('up V','Adjustment',255,255,nothing)


while(1):
    #Taking HSV values
    
    k = cv.waitKey(1) & 0xFF
    if k == 27: #esc key
        break
    # get current positions of six trackbars for lower limits
    lowH = cv.getTrackbarPos('low H','Adjustment')
    lowS = cv.getTrackbarPos('low S','Adjustment')
    lowV = cv.getTrackbarPos('low V','Adjustment')

    upH = cv.getTrackbarPos('up H','Adjustment')
    upS = cv.getTrackbarPos('up S','Adjustment')
    upV = cv.getTrackbarPos('up V','Adjustment')

    # Take each frame
    ret, frame = cap.read()

    #Flipping the frame to mirror the movements
    frame = cv.flip(frame, 1)

    #Removing noise of the frame
    filtered_frame = cv.GaussianBlur(frame,(11,11),0)

    # Convert BGR to HSV
    hsv = cv.cvtColor(filtered_frame, cv.COLOR_BGR2HSV)

    # define range of color in HSV
    lower_color = np.array([lowH,lowS,lowV])
    upper_color = np.array([upH,upS,upV])

    # Threshold the HSV image to get only the specified color
    mask = cv.inRange(hsv, lower_color, upper_color)

    kernel = np.ones((7,7),np.uint8)

    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    mask = cv.dilate(mask,kernel,iterations = 1)

    # Bitwise-AND mask and original image
    res = cv.bitwise_and(filtered_frame,filtered_frame, mask= mask)

    contours, hierarchy = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    if len(contours)!=0:

        cnt = contours[0]

        (x,y),radius = cv.minEnclosingCircle(cnt)
        center = (int(x),int(y))
        radius = int(radius)
        cv.circle(frame,center,radius,(0,255,0),2)

    cv.imshow('frame',frame)
    
    k = cv.waitKey(5) & 0xFF
    if k == 27:
        break

cap.release()
cv.destroyAllWindows()

### Making the Virtual Drawing Pad

* With the HSV values of our Stylus ready, we create the virtual drawing pad and get started scribbling!

In [48]:
#Virtual Drawing Pad

# Create a black image, a window
img = np.zeros((480,640,3), np.uint8)

cap = cv.VideoCapture(0)

i=1 #Loop variable
x_=0 #x coordinate from the Previous loop
y_=0 #y coordinate from the Previous loop

while(1):

    # Taking each frame
    ret, frame = cap.read()
    ret = cap.set(cv.CAP_PROP_FRAME_WIDTH,480)
    ret = cap.set(cv.CAP_PROP_FRAME_HEIGHT,6400)

    #Flipping the frame to mirror the movements
    frame = cv.flip(frame, 1)
    
    #Removing noise of the frame
    filtered_frame = cv.GaussianBlur(frame,(15,15),0)
    
    # Converting BGR to HSV
    hsv = cv.cvtColor(filtered_frame, cv.COLOR_BGR2HSV)

    #Thresholding the HSV image to get only specified color
    mask = cv.inRange(hsv, lower_color, upper_color)

    #Removing the small noises and openings in the mask 
    kernel = np.ones((7,7),np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    mask = cv.dilate(mask,kernel,iterations = 1)

    # Bitwise-AND mask and original image operation
    res = cv.bitwise_and(filtered_frame,filtered_frame, mask= mask)
    
    #Finding the Canny edges of the mask for better object detection
    edges = cv.Canny(mask,100,150)

    #Finding the contours of the edges
    contours, hierarchy = cv.findContours(edges, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    if len(contours)!=0: #If the object is present on the screen, len(contours)!=0

        cnt = contours[0]

        #MEnclosing the object inside a circle
        (x,y),radius = cv.minEnclosingCircle(cnt)
        center = (int(x),int(y))
        radius = int(radius)
        cv.circle(frame,center,radius,(0,255,0),2)

        #Paint application
        if i!=1:
            #Using the x,y coordinates of the circle to draw lines, with the initial points as the previous coordinates
            cv.line(img,center,(x_,y_),(0,255,255),5)
            (x_,y_)=center
        else:
            #When the object first appears, only initial coordinates are noted
            (x_,y_)=center
            i=i+1
    
    else:
        #When the object is not in the frame
        i=1
        x_=0
        y_=0
    
    
    cv.imshow('frame',frame)

    cv.imshow("Output",img)
    k = cv.waitKey(1) & 0xFF
    if k == 27: #esc key
        break

cap.release()
cv.destroyAllWindows()

# Conclusion

* Hope you liked the project and found it useful and interesting and understood the concepts which I used while making this program.

**HAPPY DRAWING HOURS!**